# The Spinning Effective One-Body Initial Condition Solver

### Author: Tyler Knowles

## This module documents the reduced spinning effective one-body initial condition solver as numerically implemented in LALSuite's SEOBNRv3 gravitational waveform approximant.  That is, we follow Section IV A of [Buonanno, Chen, and Damour (2006)](https://arxiv.org/abs/gr-qc/0508067).

### Throughout this module, we will refer to
* [Pan, Buonanno, Buchman et. al. (2010)](https://arxiv.org/abs/0912.3466v2) as PBB2010,
* [Buonanno, Chen, and Damour (2006)](https://arxiv.org/abs/gr-qc/0508067) as BCD2006, and
* [Barausse and Buonanno (2010)](https://arxiv.org/abs/0912.3517) as BB2010.

<font color='green'>**This module has been validated against the LALSuite SEOBNRv3/SEOBNRv3_opt code that was reviewed and approved for LIGO parameter estimation by the LIGO Scientific Collaboration.**</font>

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows, matching the "steps" listed in [BCD2006](https://arxiv.org/abs/gr-qc/0508067):

0. [Introduction:](#intro) The Physical System of Interest
1. [Step 1:](#step1) Initial Coordinate Choice
1. [Step 2:](#step2) Compute ${\bf r}$, ${\bf p}_{r}$, ${\bf p}_{\theta}$, and ${\bf p}_{\phi}$
1. [Step 3:](#step3) Rotate $\hat{\bf L} \to {\bf e}_{z}$
1. [Step 4:](#step4) Compute $\dot{\bf r}$
1. [Step 5:](#step5) Invert the rotation of Step 3
1. [Output](#latex_pdf_output): Output this module to $\LaTeX$-formatted PDF

<a id='intro'></a>

# Introduction: The Physical System of Interest \[Back to [top](#toc)\]
$$\label{intro}$$

Consider two compact objects (e.g. black holes or neutron stars) with masses $m_{1}$, $m_{2}$ (in solar masses) and spin angular momenta ${\bf S}_{1}$, ${\bf S}_{2}$ in a binary system.  The spinning effective one-body ("SEOB") Hamiltonian $H_{\rm real}$ (see [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.69)) describes the dynamics of this system.  We seek initial conditions for nonadiabatic evolutions of such a system, and follow [BCD2006](https://arxiv.org/abs/gr-qc/0508067) Section IV A.

Please note that throughout this notebook we adpot the following conventions:
1. $c = G = 1$ where $c$ is the speed of light in a vacuum and $G$ is Newton's gravitational constant,
1. hatted vectors (e.g. $\hat{\bf L}_{N}$) usually denote scaled or unit vectors, and
1. the initial inclination angle $\iota$ is chosen to be zero.

Choose a spatial coordinate basis $\left\{ {\bf e}_{0}, {\bf e}_{1}, {\bf e}_{2} \right\}$ so that the initial separation vector ${\bf r}$ between the compact objects lies along the ${\bf e}_{0}$-axis.  We start with the following parameters:
1. binary parameters $m_{1}$, $m_{2}$, ${\bf S}_{1}$, and ${\bf S}_{2}$,
1. initial frequency $f$, and
1. SEOB model parameters (constants dependent on $m_{1}$, $m_{2}$).

Our goal is to produce initial dynamical variables
1. ${\bf x} = \left( x, y, z \right)$, and
1. ${\bf p} = \left( p_{x}, p_{y}, p_{z} \right)$.

We include below the physical parameters necessary to compute the initial conditions.  Solving for the initial conditions requires computing $\frac{ \partial H_{\rm real} }{ \partial p_{i} }$ for $i \in \left\{ r, \theta, \phi \right\}$, <font color='red'>which we defer to another module</font>.

<font color='red'>Please note that in [BCD2006](https://arxiv.org/abs/gr-qc/0508067) the initial conditions are solved for given an initial separation; here we use a given initial frequency instead.  The difference is in our approach to solving Equation (4.8).  Our approach also differs from that found in LALSuite's XLALSimIMRSpinEOBInitialConditionsPrec() function (file: LALSimIMRSpinEOBInitialConditionsPrec.c) because we choose our intial coordinate system so that the inclination angle $\iota$ is zero (this simplifies their step one and step five).</font>

Besides the physical parameters, we also need the [Euler–Mascheroni constant](https://en.wikipedia.org/wiki/Euler%E2%80%93Mascheroni_constant) $\gamma$ and the [geomtrized](https://en.wikipedia.org/wiki/Geometrized_unit_system) solar mass $\mathcal{M}_{\odot}$, both hard-coded in LALSuite with the significant digits shown below.  (The following links are directed to the appropriate LALSuite documentation: [$\gamma$](https://lscsoft.docs.ligo.org/lalsuite/lal/group___l_a_l_constants__h.html#gac6af32574ff8acaeeafe8bf422281e98) and [$\mathcal{M}_{\odot}$](https://lscsoft.docs.ligo.org/lalsuite/lal/group___l_a_l_constants__h.html#gab83f8c705dda3fd0bb2d5f2470bb9cdd).)

In [1]:
import sympy as sp
m1,m2,S10,S11,S12,S20,S21,S22,f = sp.symbols("m1 m2 S10 S11 S12 S20 S21 S22 f",real=True)
dHdpx,dHdpy,dHdpz = sp.symbols("dHdpx dHdpy dHdpz",real=True)

gamma = 0.577215664901532860606512090082402431
Msol = 4.925491025543575903411922162094833998e-6

<a id='step1'></a>

# Step 1: Initial Coordinate Choice \[Back to [top](#toc)\]
$$\label{step1}$$

<a id='massterms'></a>

## Mass terms \[Back to [top](#toc)\]
$$\label{massterms}$$

We begin by defining the following useful quantities:

\begin{align*}
    M &= m_{1} + m_{2} \\
    \eta &= \frac{ m_{1} m_{2} }{ M^{2} }
\end{align*}

In [2]:
M = m1 + m2
Msq = M*M
eta = m1*m2/Msq

<a id='spinterms'></a>

## Spin terms \[Back to [top](#toc)\]
$$\label{spinterms}$$

<font color='red'>Can we find a source that says how the basis for the orientation of ${\bf S}_{1}$, ${\bf S}_{2}$ is chosen?  For now I assume it's in the $\hat{\bf r}, \hat{\bf v}, \hat{\bf L}_{N}$-frame, which is suggestd by the fact that LALSuite rotates them using the same rotation matrix as rotates $\hat{\bf r}$, $\hat{\bf v}$, and $\hat{\bf L}_{N}$ if $\iota \not= 0$.</font>  Since we assumed $G = c = 1$, we make the spin angular momenta dimensionless via:

\begin{align*}
    \hat{\bf S}_{1} &= \frac{ 1 }{ M^{2} } {\bf S}_{1} \\
    \hat{\bf S}_{2} &= \frac{ 1 }{ M^{2} } {\bf S}_{2}
\end{align*}

In [3]:
S1hat0 = S10/Msq
S1hat1 = S11/Msq
S1hat2 = S12/Msq
S2hat0 = S20/Msq
S2hat1 = S21/Msq
S2hat2 = S22/Msq

<a id='rhat'></a>

## Normalized Position $\hat{\bf r}$ \[Back to [top](#toc)\]
$$\label{rhat}$$

We assumed that the initial separation vector ${\bf r}$ lies along the ${\bf e}_{0}$-axis, so the normalized initial separation vector $\hat{\bf r}$ is given by

\begin{align*}
    \hat{\bf r}^{0} &= 1 \\
    \hat{\bf r}^{1} &= 0 \\
    \hat{\bf r}^{2} &= 0.
\end{align*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Lines 803--805.

In [4]:
rhat0 = 1.
rhat1 = 0.
rhat2 = 0.

<a id='ln'></a>

## Normalized Orbital Angular Momenutm $\hat{\bf L}_{N}$ \[Back to [top](#toc)\]
$$\label{ln}$$

Since we assume that the initial separation vector ${\bf r}$ between $m_{1}$ and $m_{2}$ lies along the ${\bf e}_{0}$-axis, the initial orbital plane coincides with the ${\bf e}_{0},{\bf e}_{1}$-plane.  Thus the normalized inital orbital angular momentum vector $\hat{\bf L}_{N}$ is given by

\begin{align*}
    \hat{\bf L}_{N}^{0} &= 0 \\
    \hat{\bf L}_{N}^{1} &= 0 \\
    \hat{\bf L}_{N}^{2} &= 1.
\end{align*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Lines 788--790.

In [5]:
LNhat0 = 0.
LNhat1 = 0.
LNhat2 = 1.

<a id='vhat'></a>

## Normalized Velocity $\hat{\bf v}$ \[Back to [top](#toc)\]
$$\label{vhat}$$

Given normalized orbital angular momentum ($\hat{\bf L}_{N}$) and normalized position ($\hat{\bf r}$), the normalized velocity vector ($\hat{\bf v}$) is given by

\begin{equation*}
    \hat{\bf v} = \frac{ \hat{\bf L}_{N} \times \hat{\bf r} }{ \left\lvert \hat{\bf L}_{N} \times \hat{\bf r} \right\rvert }.
\end{equation*}

Given $\hat{\bf L}_{N} = \begin{bmatrix} 0 \\ 0 \\ 1 \end{bmatrix}$ and $\hat{\bf r} = \begin{bmatrix} 1 \\ 0 \\ 0 \end{bmatrix}$ it is clear that $\hat{\bf v} = \begin{bmatrix} 0 \\ 1 \\ 0 \end{bmatrix}$.

See LALSimIMRSpinEOBInitialConditionsPrec.c Lines 812--814.

In [6]:
vhat0 = 0.
vhat1 = 1.
vhat2 = 0.

## Note \[Back to [top](#toc)\]


Since we began assuming $\iota = 0$, we do not need to rotate $\hat{\bf r}$, $\hat{\bf v}$, $\hat{\bf L}_{N}$, ${\bf S}_{1}$, ${\bf S}_{2}$, $\hat{\bf S}_{1}$, or $\hat{\bf S}_{2}$ as is done at LALSimIMRSpinEOBInitialConditionsPrec.c Lines 840.

<a id='step2'></a>

# Step 2: Compute ${\bf r}$ and ${\bf p}$ in spherical coordinates \[Back to [top](#toc)\]
$$\label{step2}$$

<a id='omega'></a>

## $\omega$ \[Back to [top](#toc)\]
$$\label{omega}$$

<font color='red'>Is there a paper reference for this formula?  It's not quite Newtonian ($\omega = 2 \pi f)$.</font>
 
\begin{equation*}
    \omega = M \mathcal{M}_{\odot} \pi f.
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 893.

In [7]:
omega = M*Msol*sp.pi*f

<a id='velocity'></a>

## Initial Velocity $v$ \[Back to [top](#toc)\]
$$\label{velocity}$$

<font color='red'>Is there a paper reference for this formula?</font>

\begin{equation*}
    v = \sqrt[3]{ \omega }.
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 894.

In [8]:
v = sp.cbrt(omega)

<a id='roots'></a>

## Root-finding...

$$\label{roots}$$

<font color='red'>**Note: LALSuite scales the initial guesses given to the root-finding routine; see LALSimIMRSpinEOBInitialConditionsPrec.c Line 899.**</font>

We will write components of the momentum vector ${\bf p}$ in spherical coordinates with components ${\bf p}^{r}$, ${\bf p}^{\theta}$, and ${\bf p}^{\phi}$.

<font color='red'>These are initial guesses for the root-finding routine:

\begin{align*}
    {\bf r}^{r} &= \frac{ 1 }{ v^{2} } \\
    {\bf p}^{\phi} &= v \\
    {\bf p}^{\theta} &= 0.2.
\end{align*}

The root finder seeks to solve

\begin{equation*}
    \begin{bmatrix} \frac{ \partial H }{ \partial {\bf r}^{r} } \\ \frac{ \partial H }{ \partial {\bf p}^{\theta} } \\ \frac{ \partial H }{ \partial {\bf p}^{\phi} } - \omega \end{bmatrix} = \begin{bmatrix} 0 \\ 0 \\ 0 \end{bmatrix}.
\end{equation*}

As the Hamiltonian is given in Cartesian coordinates, this requires computing $\frac{ \partial H }{ \partial {\bf r}^{0} }$, $\frac{ \partial H }{ \partial {\bf p}^{1} }$, and $\frac{ \partial H }{ \partial {\bf p}^{2} }$ and then converting to spherical coordinates via

\begin{align*}
    \frac{\partial H}{\partial {\bf r}^{r}} &= \frac{\partial H}{\partial {\bf r}^{0}} - \frac{\frac{\partial H}{\partial {\bf p}^{1}}{\bf p}^{\phi}}{\left({\bf r}^{r}\right)^{2}} + \frac{\frac{\partial H}{\partial {\bf p}^{2}}{\bf p}^{\theta}}{\left({\bf r}^{r}\right)^{2}} \\
    \frac{\partial H}{\partial {\bf p}^{\theta}} &= -\frac{\frac{\partial H}{\partial {\bf p}^{2}}}{{\bf r}^{r}} \\
    \frac{\partial H}{\partial {\bf p}^{\phi}} &= \frac{\frac{\partial H}{\partial {\bf p}^{1}}}{{\bf r}^{r}}.
\end{align*}

In the end, we should have a cartesian postition vector ${\bf q}$ and momentum vector ${\bf p}$.  It seems that because we stared with $\iota = 0$ that we can use $\hat{\bf r}$ instead of $\hat{\bf q}$?</font>

In [9]:
q0 = 1.
q1 = 1.
q2 = 1.
p0 = 1.
p1 = 1.
p2 = 1.

<a id='step3'></a>

# Step 3: Rotate $\hat{\bf L} \to {\bf e}_{z}$ \[Back to [top](#toc)\]
$$\label{step3}$$

## Note \[Back to [top](#toc)\]


At this point, see LALSimIMRSpinEOBInitialConditionsPrec.c normalizes the Cartesian separation and momentum vectors constructed in [Step 2](#step2).  We already have a normalized separation vector $\hat{\bf r}$, so we skip that step.

<a id='normp'></a>

## Normalize ${\bf p}$ \[Back to [top](#toc)\]
$$\label{normp}$$

Next we normalize the new position vector ${\bf p}$ that we found in [Step 2](#step2):

\begin{equation*}
    \hat{\bf p} = \frac{ {\bf p} }{ \left\lvert {\bf p} \right\rvert}.
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 1101.

In [10]:
pnorminv = 1/sp.sqrt(p0*p0 + p1*p1 + p2*p2)
phat0 = p0*pnorminv
phat1 = p1*pnorminv
phat2 = p2*pnorminv

<a id='lhat'></a>

## $\hat{\bf L}$ \[Back to [top](#toc)\]
$$\label{lhat}$$

We compute the normalized relativistic angular momentum vector $\hat{\bf L}$ (<font color='red'>find source for this formula?</font>):

\begin{equation*}
    \hat{\bf L} = \frac{ \hat{\bf r} \times \hat{\bf p} }{ \left\lvert \hat{\bf r} \times \hat{\bf p} \right\rvert }.
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Lines 1098--1100.

In [11]:
Lhat0 = rhat1*phat2 - rhat2*phat1
Lhat1 = rhat2*phat0 - rhat0*phat2
Lhat2 = rhat0*phat1 - rhat1*phat0
Lhatnorminv = 1./sp.sqrt(Lhat0*Lhat0 + Lhat1*Lhat1 + Lhat2*Lhat2)
Lhat0 /= Lhatnorminv
Lhat1 /= Lhatnorminv
Lhat2 /= Lhatnorminv

<a id='rotationmatrix'></a>

## Rotation matrix \[Back to [top](#toc)\]
$$\label{rotationmatrix}$$

The rotation matrix from the $\left\{ \hat{\bf r}, {\bf v}, \hat{\bf L}_{N} \right\}$ frame to the $\left\{ \hat{\bf r}, {\bf p}, \hat{\bf L} \right\}$ frame is given by

\begin{equation*}
    \begin{bmatrix} \hat{\bf r}^{0} & \hat{\bf r}^{1} & \hat{\bf r}^{2} \\
        \hat{\bf p}^{0} & \hat{\bf p}^{1} & \hat{\bf p}^{2} \\
        \hat{\bf L}^{0} & \hat{\bf L}^{1} & \hat{\bf L}^{2}\end{bmatrix}.
\end{equation*}

<font color='red'>The matrix to invert the rotation is simply the transpose:</font>

\begin{equation*}
    \begin{bmatrix} \hat{\bf r}^{0} & \hat{\bf p}^{0} & \hat{\bf L}^{0} \\
        \hat{\bf r}^{1} & \hat{\bf p}^{1} & \hat{\bf L}^{1} \\
        \hat{\bf r}^{2} & \hat{\bf p}^{2} & \hat{\bf L}^{2}\end{bmatrix}.
\end{equation*}

To see that this is indeed the correct matrix inverse, note that by construction $\hat{\bf q}$, $\hat{\bf p}$, and $\hat{\bf L}$ are all unit vectors orthogonal to one another.  See LALSimIMRSpinEOBInitialConditionsPrec.c Line 1107.

In [12]:
rotate00 = rhat0
rotate01 = rhat1
rotate02 = rhat2
rotate10 = phat0
rotate11 = phat1
rotate12 = phat2
rotate20 = Lhat0
rotate21 = Lhat1
rotate22 = Lhat2

<a id='rotaterhat'></a>

## Rotate $\hat{\bf r}$ \[Back to [top](#toc)\]
$$\label{rotatesrhat}$$

We now rotate $\hat{\bf r}$.  We'll use primes to denote the rotated vector.

\begin{equation*}
    \hat{\bf r}^{\prime} = \begin{bmatrix} \hat{\bf r}^{0} & \hat{\bf r}^{1} & \hat{\bf r}^{2} \\
        \hat{\bf p}^{0} & \hat{\bf p}^{1} & \hat{\bf p}^{2} \\
        \hat{\bf L}^{0} & \hat{\bf L}^{1} & \hat{\bf L}^{2}\end{bmatrix}
        \begin{bmatrix} \hat{\bf r}^{0} \\ \hat{\bf r}^{1} \\ \hat{\bf r}^{2} \end{bmatrix}
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 1112.

In [13]:
rhatprm0 = rhat0*rhat0 + rhat1*rhat1 + rhat2*rhat2
rhatprm1 = phat0*rhat0 + phat1*rhat1 + phat2*rhat2
rhatprm2 = Lhat0*rhat0 + Lhat1*rhat1 + Lhat2*rhat2

<a id='rotatevhat'></a>

## Rotate $\hat{\bf v}$ \[Back to [top](#toc)\]
$$\label{rotatevhat}$$

We rotate $\hat{\bf v}$.  We'll use primes to denote the rotated vector.

\begin{equation*}
    \hat{\bf v}^{\prime} = \begin{bmatrix} \hat{\bf r}^{0} & \hat{\bf r}^{1} & \hat{\bf r}^{2} \\
        \hat{\bf p}^{0} & \hat{\bf p}^{1} & \hat{\bf p}^{2} \\
        \hat{\bf L}^{0} & \hat{\bf L}^{1} & \hat{\bf L}^{2}\end{bmatrix}
        \begin{bmatrix} \hat{\bf v}^{0} \\ \hat{\bf v}^{1} \\ \hat{\bf v}^{2} \end{bmatrix}
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 1113.

In [14]:
vhatprm0 = rhat0*vhat0 + rhat1*vhat1 + rhat2*vhat2
vhatprm1 = phat0*vhat0 + phat1*vhat1 + phat2*vhat2
vhatprm2 = Lhat0*vhat0 + Lhat1*vhat1 + Lhat2*vhat2

<a id='rotatelnhat'></a>

## Rotate $\hat{\bf L}_{N}$ \[Back to [top](#toc)\]
$$\label{rotatelnhat}$$

We rotate $\hat{\bf L}_{N}$.  We'll use primes to denote the rotated vector.

\begin{equation*}
    \hat{\bf L}_{N}^{\prime} = \begin{bmatrix} \hat{\bf r}^{0} & \hat{\bf r}^{1} & \hat{\bf r}^{2} \\
        \hat{\bf p}^{0} & \hat{\bf p}^{1} & \hat{\bf p}^{2} \\
        \hat{\bf L}^{0} & \hat{\bf L}^{1} & \hat{\bf L}^{2}\end{bmatrix}
        \begin{bmatrix} \hat{\bf L}_{N}^{0} \\ \hat{\bf L}_{N}^{1} \\ \hat{\bf L}_{N}^{2} \end{bmatrix}
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 1114.

In [15]:
LNhatprm0 = rhat0*LNhat0 + rhat1*LNhat1 + rhat2*LNhat2
LNhatprm1 = phat0*LNhat0 + phat1*LNhat1 + phat2*LNhat2
LNhatprm2 = Lhat0*LNhat0 + Lhat1*LNhat1 + Lhat2*LNhat2

<a id='rotates1'></a>

## Rotate ${\bf S}_{1}$ \[Back to [top](#toc)\]
$$\label{rotates1}$$

We rotate ${\bf S}_{1}$.  We'll use primes to denote the rotated vector.

\begin{equation*}
    {\bf S}_{1}^{\prime} = \begin{bmatrix} \hat{\bf r}^{0} & \hat{\bf r}^{1} & \hat{\bf r}^{2} \\
        \hat{\bf p}^{0} & \hat{\bf p}^{1} & \hat{\bf p}^{2} \\
        \hat{\bf L}^{0} & \hat{\bf L}^{1} & \hat{\bf L}^{2}\end{bmatrix}
        \begin{bmatrix} {\bf S}_{1}^{0} \\ {\bf S}_{1}^{1} \\ {\bf S}_{1}^{2} \end{bmatrix}
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 1115.

In [16]:
S1prm0 = rhat0*S10 + rhat1*S11 + rhat2*S12
S1prm1 = phat0*S10 + phat1*S11 + phat2*S12
S1prm2 = Lhat0*S10 + Lhat1*S11 + Lhat2*S12

<a id='rotates2'></a>

## Rotate ${\bf S}_{2}$ \[Back to [top](#toc)\]
$$\label{rotates2}$$

We rotate ${\bf S}_{2}$.  We'll use primes to denote the rotated vector.

\begin{equation*}
    {\bf S}_{2}^{\prime} = \begin{bmatrix} \hat{\bf r}^{0} & \hat{\bf r}^{1} & \hat{\bf r}^{2} \\
        \hat{\bf p}^{0} & \hat{\bf p}^{1} & \hat{\bf p}^{2} \\
        \hat{\bf L}^{0} & \hat{\bf L}^{1} & \hat{\bf L}^{2}\end{bmatrix}
        \begin{bmatrix} {\bf S}_{2}^{0} \\ {\bf S}_{2}^{1} \\ {\bf S}_{2}^{z} \end{bmatrix}
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 1116.

In [17]:
S2prm0 = rhat0*S20 + rhat1*S21 + rhat2*S22
S2prm1 = phat0*S20 + phat1*S21 + phat2*S22
S2prm2 = Lhat0*S20 + Lhat1*S21 + Lhat2*S22

<a id='rotates1hat'></a>

## Rotate $\hat{\bf S}_{1}$ \[Back to [top](#toc)\]
$$\label{rotates1hat}$$

We rotate $\hat{\bf S}_{1}$.  We'll use primes to denote the rotated vector.

\begin{equation*}
    \hat{\bf S}_{1}^{\prime} = \begin{bmatrix} \hat{\bf r}^{0} & \hat{\bf r}^{1} & \hat{\bf r}^{2} \\
        \hat{\bf p}^{0} & \hat{\bf p}^{1} & \hat{\bf p}^{2} \\
        \hat{\bf L}^{0} & \hat{\bf L}^{1} & \hat{\bf L}^{2}\end{bmatrix}
        \begin{bmatrix} \hat{\bf S}_{1}^{0} \\ \hat{\bf S}_{1}^{1} \\ \hat{\bf S}_{1}^{1} \end{bmatrix}
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 1117.

In [18]:
S1hatprm0 = rhat0*S1hat0 + rhat1*S1hat1 + rhat2*S1hat2
S1hatprm1 = phat0*S1hat0 + phat1*S1hat1 + phat2*S1hat2
S1hatprm2 = Lhat0*S1hat0 + Lhat1*S1hat1 + Lhat2*S1hat2

<a id='rotates2hat'></a>

## Rotate $\hat{\bf S}_{2}$ \[Back to [top](#toc)\]
$$\label{rotates2hat\hat}$$

We rotate $\hat{\bf S}_{2}$.  We'll use primes to denote the rotated vector.

\begin{equation*}
    \hat{\bf S}_{2}^{\prime} = \begin{bmatrix} \hat{\bf r}^{0} & \hat{\bf r}^{1} & \hat{\bf r}^{2} \\
        \hat{\bf p}^{0} & \hat{\bf p}^{1} & \hat{\bf p}^{2} \\
        \hat{\bf L}^{0} & \hat{\bf L}^{1} & \hat{\bf L}^{2}\end{bmatrix}
        \begin{bmatrix} \hat{\bf S}_{2}^{0} \\ \hat{\bf S}_{2}^{1} \\ \hat{\bf S}_{2}^{2} \end{bmatrix}
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 1118.

In [19]:
S2hatprm0 = rhat0*S2hat0 + rhat1*S2hat1 + rhat2*S2hat2
S2hatprm1 = phat0*S2hat0 + phat1*S2hat1 + phat2*S2hat2
S2hatprm2 = Lhat0*S2hat0 + Lhat1*S2hat1 + Lhat2*S2hat2

<a id='rotater'></a>

## Rotate ${\bf r}$ \[Back to [top](#toc)\]
$$\label{rotater}$$

We rotate ${\bf p}$.  We'll use primes to denote the rotated vector.

\begin{equation*}
    {\bf r}^{\prime} = \begin{bmatrix} \hat{\bf r}^{0} & \hat{\bf r}^{1} & \hat{\bf r}^{2} \\
        \hat{\bf p}^{0} & \hat{\bf p}^{1} & \hat{\bf p}^{2} \\
        \hat{\bf L}^{0} & \hat{\bf L}^{1} & \hat{\bf L}^{2}\end{bmatrix}
        \begin{bmatrix} {\bf r}^{0} \\ {\bf r}^{1} \\ {\bf r}^{2} \end{bmatrix}
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 1119.

In [20]:
rprm0 = rhat0*p0 + rhat1*p1 + rhat2*p2
rprm1 = phat0*p0 + phat1*p1 + phat2*p2
rprm2 = Lhat0*p0 + Lhat1*p1 + Lhat2*p2

<a id='rotatep'></a>

## Rotate ${\bf p}$ \[Back to [top](#toc)\]
$$\label{rotatep}$$

We rotate ${\bf p}$.  We'll use primes to denote the rotated vector.

\begin{equation*}
    {\bf p}^{\prime} = \begin{bmatrix} \hat{\bf r}^{0} & \hat{\bf r}^{1} & \hat{\bf r}^{2} \\
        \hat{\bf p}^{0} & \hat{\bf p}^{1} & \hat{\bf p}^{2} \\
        \hat{\bf L}^{0} & \hat{\bf L}^{1} & \hat{\bf L}^{2}\end{bmatrix}
        \begin{bmatrix} {\bf p}^{0} \\ {\bf p}^{y} \\ {\bf p}^{2} \end{bmatrix}
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 1120.

In [21]:
pprm0 = rhat0*p0 + rhat1*p1 + rhat2*p2
pprm1 = phat0*p0 + phat1*p1 + phat2*p2
pprm2 = Lhat0*p0 + Lhat1*p1 + Lhat2*p2

<a id='step4'></a>

# Step 4: Compute $\dot{\bf r}$ \[Back to [top](#toc)\]
$$\label{step4}$$

<a id='carttosph'></a>

## Convert from Cartesian to Spherical \[Back to [top](#toc)\]
$$\label{carttosph}$$

We convert position and momentum into spherical coordinates.  In the special case where $\theta = \frac{ \pi }{ 2 }$ and $\phi = 0$, the spherical position vector ${\bf r} = \left( {\bf r}^{r}, {\bf r}^{\theta}, {\bf r}^{\phi} \right)$ is given by

\begin{align*}
    {\bf r}^{r} &= {\bf r}^{0} \\
    {\bf r}^{\theta} &= \frac{ \pi }{ 2 } \\
    {\bf r}^{\phi} &= 0
\end{align*}

and the spherical momentum vector ${\bf p} = \left( {\bf p}^{r}, {\bf p}^{\theta}, {\bf p}^{\phi} \right)$ is given by

\begin{align*}
    {\bf p}^{r} &= {\bf p}^{0} \\
    {\bf p}^{\theta} &= - {\bf r}^{0}{\bf p}^{2} \\
    {\bf p}^{\phi} &= {\bf r}^{0}{\bf p}^{1} \\
\end{align*}

We call a Cartesian to spherical routine at LALSimIMRSpinEOBInitialConditionsPrec.c Line 1139, and the function itself is defined on Lines 243--285.

In [22]:
rr = rprm0
rtheta = sp.pi/2.
rphi = 0.
pr = pprm0
ptheta = -rprm0*pprm2
pphi = rprm0*pprm1

<a id='secondderiv'></a>

## Second derivatives of $H_{\rm real}$ \[Back to [top](#toc)\]
$$\label{seconderiv}$$

We need to compute $\frac{ \partial H }{ \partial {\bf p}^{\phi} }$, $\frac{ \partial^{2} H_{\rm real} }{ \partial r^{2} }$, and $\frac{ \partial^{2} H_{\rm real} }{ \partial r \partial {\bf p}^{\phi} }$ (<font color='red'>in another module</font>).

<font color='red'>Note: be sure that, following this, we use normalized spins.</font>

In [23]:
dHdpphi = 0.
d2Hdr2 = 0.
d2Hdrdpphi = 1.

<a id='dEdr'></a>

## $\frac{ \partial E }{ \partial r }$ \[Back to [top](#toc)\]
$$\label{dEdr}$$

I don't know what this is or means (see [BCD2006](https://arxiv.org/abs/gr-qc/0508067) Equations (4.14) and (3.7)), but we compute $\frac{ \partial E }{ \partial r }$ in spherical coordinates (when ${\bf r}$ is directed along the ${\bf e}_{0}$ axis) via

\begin{equation*}
    \frac{ \partial E }{ \partial r } = -\frac{ \frac{ \partial H }{ \partial {\bf p}^{\phi} } \frac{ \partial^{2} H }{ \left(\partial {\bf r}^{r} \right)^{2} } }{ \frac{ \partial^{2} H }{ \partial H \partial {\bf p}^{\phi} } }.
\end{equation*}

<font color='red'></font>

In [24]:
dEdr = -dHdpphi*d2Hdr2/d2Hdrdpphi

<a id='skerr'></a>

## ${\bf S}_{\rm Kerr}$ \[Back to [top](#toc)\]
$$\label{skerr}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equations (5.2), (5.63), and (5.67) we have

\begin{equation*}
    {\bf S}_{\rm Kerr} = {\bf S}_{1} + {\bf S}_{2}.
\end{equation*}

Taking the square of [BB2010](https://arxiv.org/abs/0912.3517) Equation (4.9) <font color='red'>(be careful with the factor or $M$...)</font>,

\begin{equation*}
    a^{2} = \frac{ {\bf S}_{Kerr} \cdot {\bf S}_{Kerr} }{ M^{2} }
\end{equation*}

so that

\begin{equation*}
    a = \sqrt{ a^{2} }.
\end{equation*}

In [25]:
SKerr0 = S10 + S20
SKerr1 = S11 + S21
SKerr2 = S12 + S22
asq = (SKerr0*SKerr0 + SKerr1*SKerr1 + SKerr2*SKerr2)/(M*M)
a = sp.sqrt(asq)

<a id='sigmastar'></a>

## $\boldsymbol{\sigma}^{*}$ \[Back to [top](#toc)\]
$$\label{sigmastar}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.3),

\begin{equation*}
    \boldsymbol{\sigma}^{*} = \frac{ m_{2} }{ m_{1} } {\bf S}_{1} + \frac{ m_{1} }{ m_{2} }{\bf S}_{2}.
\end{equation*}

In [26]:
sigmastar0 = S10*m2/m1 + S20*m1/m2
sigmastar1 = S11*m2/m1 + S21*m1/m2
sigmastar2 = S12*m2/m1 + S22*m1/m2

<a id='Hreal'></a>

## $H_{\rm real}$ \[Back to [top](#toc)\]
$$\label{Hreal}$$

We now compute $H_{\rm real}$ (LALSimIMRSpinEOBInitialConditionsPrec.c Line 1217) (<font color='red'>another module</font>).

In [27]:
Hreal = 0

<a id='polardata'></a>

## Polar data \[Back to [top](#toc)\]
$$\label{polardata}$$

At LALSimIMRSpinEOBInitialConditionsPrec.c Lines 1234--1238, we set the following polar data ${\bf P}$ <font color='red'>(need to find reference for this?)</font>:

\begin{align*}
    {\bf P}^{0} &= {\bf r}^{r} \\
    {\bf P}^{1} &= 0 \\
    {\bf P}^{2} &= {\bf p}^{r} \\
    {\bf P}^{3} &= {\bf p}^{\phi}
\end{align*}

In [28]:
polar0 = rr
polar1 = 0.
polar2 = pr
polar3 = pphi

<a id='l'></a>

## Compute ${\bf L}$ \[Back to [top](#toc)\]
$$\label{l}$$

We compute ${\bf L} = {\bf r}^{\prime}\times{\bf p}^{\prime}$ and normalize as follows.

\begin{equation*}
    {\bf L} = \frac{ {\bf r}^{\prime} \times {\bf p}^{\prime} }{ \left\lvert {\bf r}^{\prime} \times {\bf p}^{\prime} \right\rvert }
\end{equation*}

See LALSimIMRSpinEOBFactorizedFluxPrec_v3opt.c lines 121--129.

In [ ]:
L0 = rprm1*pprm2 - rprm2*pprm1
L1 = rprm2*pprm0 - rprm0*pprm2
L2 = rprm0*pprm1 - rprm1*pprm0
Lmaginv = 1./sp.sqrt(rprmcrosspprm0*rprmcrosspprm0
                                 + rprmcrosspprm1*rprmcrosspprm1
                                 + rprmcrosspprm2*rprmcrosspprm2)
L0 /= Lmaginv
L1 /= Lmaginv
L2 /= Lmaginv

<a id='s1dot'></a>

## ${\bf S}_{1}\cdot{\bf L}$ \[Back to [top](#toc)\]
$$\label{s1dotl}$$

We compute ${\bf r}^{\prime}\times{\bf p}^{\prime}$ and normalize as follows.

See LALSimIMRSpinEOBFactorizedFluxPrec_v3opt.c lines 121--129.

<a id='step5'></a>

# Step 5: Invert the rotation of Step 3 \[Back to [top](#toc)\]
$$\label{step5}$$

<a id='invrotationmatrix'></a>

## Inverse Rotation Matrix \[Back to [top](#toc)\]
$$\label{invrotationmatrix}$$

The matrix to invert the rotation applied in [Step 3](#step3) is:

\begin{equation*}
    \begin{bmatrix} \hat{\bf r}^{0} & \hat{\bf p}^{0} & \hat{\bf L}^{0} \\
        \hat{\bf r}^{1} & \hat{\bf p}^{1} & \hat{\bf L}^{1} \\
        \hat{\bf r}^{2} & \hat{\bf p}^{2} & \hat{\bf L}^{2}\end{bmatrix}.
\end{equation*}

To see that this is indeed the correct matrix inverse, note that by construction $\hat{\bf q}$, $\hat{\bf p}$, and $\hat{\bf L}$ are all unit vectors orthogonal to one another.  See LALSimIMRSpinEOBInitialConditionsPrec.c Line 1107.

In [29]:
invert00 = rhat0
invert01 = phat0
invert02 = Lhat0
invert10 = rhat1
invert11 = phat1
invert12 = Lhat1
invert20 = rhat2
invert21 = phat2
invert22 = Lhat2

<a id='invrotaterhat'></a>

## Rotate $\hat{\bf r}^{\prime}$ \[Back to [top](#toc)\]
$$\label{invrotaterhat}$$

We rotate $\hat{\bf r}^{\prime}$ and call the new separation vector ${\bf r}$.

\begin{equation*}
    \hat{\bf r} = \begin{bmatrix} \hat{\bf r}^{0} & \hat{\bf p}^{0} & \hat{\bf L}^{0} \\
        \hat{\bf r}^{1} & \hat{\bf p}^{1} & \hat{\bf L}^{1} \\
        \hat{\bf r}^{2} & \hat{\bf p}^{2} & \hat{\bf L}^{2} \end{bmatrix}
        \begin{bmatrix} \hat{\bf r}^{\prime 0} \\ \hat{\bf r}^{\prime 1} \\ \hat{\bf r}^{\prime 2} \end{bmatrix}
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 1315.

In [30]:
rhat0 = rhat0*rhatprm0 + phat0*rhatprm1 + Lhat0*rhatprm2
rhat1 = rhat1*rhatprm0 + phat1*rhatprm1 + Lhat1*rhatprm2
rhat0 = rhat2*rhatprm0 + phat2*rhatprm1 + Lhat2*rhatprm2

<a id='invrotatevhat'></a>

## Rotate $\hat{\bf v}^{\prime}$ \[Back to [top](#toc)\]
$$\label{invrotatevhat}$$

We rotate $\hat{\bf v}^{\prime}$ and call the new separation vector ${\bf v}$.

\begin{equation*}
    \hat{\bf v} = \begin{bmatrix} \hat{\bf r}^{0} & \hat{\bf p}^{0} & \hat{\bf L}^{0} \\
        \hat{\bf r}^{1} & \hat{\bf p}^{1} & \hat{\bf L}^{1} \\
        \hat{\bf r}^{2} & \hat{\bf p}^{2} & \hat{\bf L}^{2} \end{bmatrix}
        \begin{bmatrix} \hat{\bf v}^{\prime 0} \\ \hat{\bf v}^{\prime 1} \\ \hat{\bf v}^{\prime 2} \end{bmatrix}
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 1316.

In [31]:
vhat0 = rhat0*vhatprm0 + phat0*vhatprm1 + Lhat0*vhatprm2
vhat1 = rhat1*vhatprm0 + phat1*vhatprm1 + Lhat1*vhatprm2
vhat2 = rhat2*vhatprm0 + phat2*vhatprm1 + Lhat2*vhatprm2

<a id='invrotatelnhat'></a>

## Rotate $\hat{\bf L}_{N}^{\prime}$ \[Back to [top](#toc)\]
$$\label{invrotatelnhat}$$

We rotate $\hat{\bf L}_{N}^{\prime}$ and call the new separation vector ${\bf L}_{N}$.

\begin{equation*}
    \hat{\bf L}_{N} = \begin{bmatrix} \hat{\bf r}^{0} & \hat{\bf p}^{0} & \hat{\bf L}^{0} \\
        \hat{\bf r}^{1} & \hat{\bf p}^{1} & \hat{\bf L}^{1} \\
        \hat{\bf r}^{2} & \hat{\bf p}^{2} & \hat{\bf L}^{2} \end{bmatrix}
        \begin{bmatrix} \hat{\bf L}_{N}^{\prime 0} \\ \hat{\bf L}_{N}^{\prime 1} \\ \hat{\bf L}_{N}^{\prime 2} \end{bmatrix}
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 1317.

In [32]:
LNhat0 = rhat0*LNhatprm0 + phat0*LNhatprm1 + Lhat0*LNhatprm2
LNhat1 = rhat1*LNhatprm0 + phat1*LNhatprm1 + Lhat1*LNhatprm2
LNhat2 = rhat2*LNhatprm0 + phat2*LNhatprm1 + Lhat2*LNhatprm2

<a id='tortoise_matrix'></a>

# Tortoise Conversion Matrix \[Back to [top](#toc)\]
$$\label{tortoise_matrix}$$

<font color='red'>We're now back to LALSpinPrecHcapRvecDerivative_v3opt.c, Lines 92--96.</font>

From [Pan, Buonanno, Buchman, et. al. (2010)](https://arxiv.org/abs/0912.3466v2) Equation (A3) the matrix for the coordinate conversion to tortoise coordinates is

\begin{align*}
    \begin{pmatrix} 1 + \frac{ x^{2} }{ r^{2} } \left( \xi - 1 \right) & \frac{ x y }{ r^{2} } \left( \xi - 1 \right) & \frac{ x z }{ r^{2} } \left( \xi - 1 \right) \\
        \frac{ x y }{ r^{2} } \left( \xi - 1 \right) & 1 + \frac{ y^{2} }{ r^{2} } \left( \xi - 1 \right) & \frac{ y z }{ r^{2} } \left( \xi - 1 \right) \\
        \frac{ x z }{ r^{2} } \left( \xi - 1 \right) & \frac{ y z }{ r^{2} } \left( \xi - 1 \right) & 1 + \frac{ z^{2} }{ r^{2} } \left( \xi - 1 \right) \end{pmatrix}
\end{align*}

In [33]:
ximinus1 = xi - 1
toTort = sp.Array([[1 + x*x*ximinus1/(r*r), x*y*ximinus1/(r*r), x*z*ximinus1/(r*r)],
                   [x*y*ximinus1/(r*r), 1 + y*y*ximinus1/(r*r), y*z*ximinus1/(r*r)],
                   [x*z*ximinus1/(r*r), y*z*ximinus1/(r*r), 1 + z*z*ximinus1/(r*r)]])

NameError: global name 'xi' is not defined

<a id='latex_pdf_output'></a>

# Output: Output this module to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

In [ ]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-SEOBNR_Initial_Conditions.ipynb
!pdflatex -interaction=batchmode Tutorial-SEOBNR_Initial_Conditions.tex
!pdflatex -interaction=batchmode Tutorial-SEOBNR_Initial_Conditions.tex
!pdflatex -interaction=batchmode Tutorial-SEOBNR_Initial_Conditions.tex
!rm -f Tut*.out Tut*.aux Tut*.log